In [25]:
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, timedelta
import numpy as np

sns.set_style("darkgrid")
sns.set(font_scale=1.5)
%matplotlib inline

In [26]:
def query_data(query, parms=None):
    """Shared query wrapper"""
    engine = create_engine('sqlite:///./../../data/processed/airlines.db')
    with engine.connect() as conn:
        data = pd.read_sql(sql=query, con=conn, params=parms)
    return data

def round_to_previous_hour(ts):
    """Revert a timestamp to the last full hour, for use with pd.apply()"""
    return ts.replace(microsecond=0, second=0, minute=0)

def back_one_hour(ts):
    """Return a timestamp a set number of hours in past"""
    return ts - timedelta(hours=1)

In [27]:
def query_departures(airport='ATL', start_date=None, end_date=None):
    """Query departure data from flights for selected origin airport and year"""
    if start_date is None:
        start_date = date(2017, 1, 1)
    if end_date is None:
        end_date = date(2017, 12, 31)
    
    query = """
    SELECT
        f.departure_was_delayed_15,
        f.year,
        f.month,
        f.day_of_week,
        f.origin,
        f.dest,
        f.departure_time_scheduled as scheduled_departure_time,
        f.distance as flight_distance,
        f.elapsed_time_scheduled as scheduled_travel_time
    FROM
        flights AS f
    WHERE
        f.Origin = :airport
    AND
        f.flight_date
    BETWEEN :start_date
        AND :end_date
    """
    data = query_data(query, parms={'airport': airport, 'start_date':start_date, 'end_date':end_date})
    # Build datetime elements for merging
    data['scheduled_departure_time'] = data['scheduled_departure_time'].apply(pd.to_datetime)
    data['hour_of_day'] = data['scheduled_departure_time'].dt.hour
    data['departure_hour'] = data['scheduled_departure_time'].apply(round_to_previous_hour)
    data['departure_hour_minus_one'] = data['departure_hour'].apply(back_one_hour)
    data['departure_hour_minus_two'] = data['departure_hour_minus_one'].apply(back_one_hour)
    data['departure_hour_minus_three'] = data['departure_hour_minus_two'].apply(back_one_hour)
    return data

departures = query_departures()
departures.head()

,departure_was_delayed_15,year,month,day_of_week,origin,dest,scheduled_departure_time,flight_distance,scheduled_travel_time,hour_of_day,departure_hour,departure_hour_minus_one,departure_hour_minus_two,departure_hour_minus_three
0,0,2017,9,2,ATL,BOS,2017-09-26 09:30:00,946,165,9,2017-09-26 09:00:00,2017-09-26 08:00:00,2017-09-26 07:00:00,2017-09-26 06:00:00
1,0,2017,9,2,ATL,BOS,2017-09-26 12:47:00,946,165,12,2017-09-26 12:00:00,2017-09-26 11:00:00,2017-09-26 10:00:00,2017-09-26 09:00:00
2,0,2017,9,2,ATL,BOS,2017-09-26 17:25:00,946,171,17,2017-09-26 17:00:00,2017-09-26 16:00:00,2017-09-26 15:00:00,2017-09-26 14:00:00
3,0,2017,9,2,ATL,BOS,2017-09-26 19:56:00,946,169,19,2017-09-26 19:00:00,2017-09-26 18:00:00,2017-09-26 17:00:00,2017-09-26 16:00:00
4,0,2017,9,2,ATL,BOS,2017-09-26 07:00:00,946,165,7,2017-09-26 07:00:00,2017-09-26 06:00:00,2017-09-26 05:00:00,2017-09-26 04:00:00


In [28]:
def query_weather(station_id='WBAN:13874', start_date=None, end_date=None):
    """Query weather data based on station_id"""
    if start_date is None:
        start_date = date(2017, 1, 1)
    if end_date is None:
        end_date = date(2017, 12, 31)
    
    query = """
    SELECT
        w.station,
        w.station_name,
        w.date,
        w.hourly_visibility as visibility,
        w.hourly_dry_bulb_temp_f as temperature_f,
        w.hourly_precipitation as precipitation,
        w.hourly_wind_speed as average_windspeed,
        w.hourly_wind_gust_speed as maximum_gust_speed,
        w.hourly_station_pressure as station_pressure
    FROM
        weather AS w
    WHERE
        w.station = :station_id
    AND
        w.date
    BETWEEN :start_date
        AND :end_date
    """
    data = query_data(query, parms={'station_id':station_id, 'start_date':start_date, 'end_date':end_date})
    
    # Handle prep for timestamp features
    data['date'] = data['date'].apply(pd.to_datetime)
    data['measurement_hour'] = data['date'].apply(round_to_previous_hour)
    return data

weather = query_weather()
weather.head()

,station,station_name,date,visibility,temperature_f,precipitation,average_windspeed,maximum_gust_speed,station_pressure,measurement_hour
0,WBAN:13874,ATLANTA HARTSFIELD INTERNATIONAL AIRPORT GA US,2017-01-01 00:52:00,5.0,42.0,0.05,11.0,17.0,28.98,2017-01-01 00:00:00
1,WBAN:13874,ATLANTA HARTSFIELD INTERNATIONAL AIRPORT GA US,2017-01-01 01:07:00,5.0,43.0,0.05,15.0,0.0,28.96,2017-01-01 01:00:00
2,WBAN:13874,ATLANTA HARTSFIELD INTERNATIONAL AIRPORT GA US,2017-01-01 01:18:00,5.0,43.0,0.05,22.0,26.0,28.92,2017-01-01 01:00:00
3,WBAN:13874,ATLANTA HARTSFIELD INTERNATIONAL AIRPORT GA US,2017-01-01 01:40:00,5.0,43.0,0.00,9.0,0.0,28.95,2017-01-01 01:00:00
4,WBAN:13874,ATLANTA HARTSFIELD INTERNATIONAL AIRPORT GA US,2017-01-01 01:52:00,6.0,44.0,0.04,9.0,0.0,28.95,2017-01-01 01:00:00


In [30]:
def prepare_features_for_analysis(departures, weather):
    data = departures.merge(right=weather, left_on='departure_hour', right_on='measurement_hour', suffixes=("", "_minus_one"))
    data = departures.merge(right=weather, left_on='departure_hour', right_on='measurement_hour', suffixes=("", "_minus_one"))
    data = data.merge(right=weather, left_on='departure_hour_minus_two', right_on='measurement_hour', suffixes=("", "_minus_two"))
    data = data.merge(right=weather, left_on='departure_hour_minus_three', right_on='measurement_hour', suffixes=("", "_minus_three"))
   
    return  data.select_dtypes(include=[np.number])

features = prepare_features_for_analysis(departures, weather)

In [31]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1525098 entries, 0 to 1525097
Data columns (total 25 columns):
departure_was_delayed_15          1525098 non-null int64
year                              1525098 non-null int64
month                             1525098 non-null int64
day_of_week                       1525098 non-null int64
flight_distance                   1525098 non-null int64
scheduled_travel_time             1525098 non-null int64
hour_of_day                       1525098 non-null int64
visibility                        1525098 non-null float64
temperature_f                     1525098 non-null float64
precipitation                     1525098 non-null float64
average_windspeed                 1525098 non-null float64
maximum_gust_speed                1525098 non-null float64
station_pressure                  1525098 non-null float64
visibility_minus_two              1525098 non-null float64
temperature_f_minus_two           1525098 non-null float64
precipitation_mi

# To what extent are there correlations in the existing features?

In [ ]:
def plot_correlation_heatmap(correlation_matrix, save_image=False):
    fig, ax = plt.subplots(figsize=(11.7, 8.27), dpi=240)

    plt.title("Pearson Correlation For Selected Features")
    sns.heatmap(correlation_matrix, ax=ax, annot=True,  cmap="YlGnBu")
    if save_image:
        plt.savefig("../../reports/figures/initial-feature-set-correlation.png", bbox_inches='tight')
    plt.show()
    
plot_correlation_heatmap(features.corr(), save_image=True)

There High levels of correlation between elapsed time and flight distance (i.e. a flight that travels further will be in the air longer.)

In [ ]:
y = features['was_delayed']
X = features.drop(['was_delayed', 'flight_distance'], axis=1)

In [ ]:
## generate a plot of gini importances with and without balance classes, showing that even that one small change can have an impact on what is important

from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd

def estimate_feature_importances(X, y, feature_names, class_weight=None):
    forest = ExtraTreesClassifier(n_estimators=250, random_state=0, class_weight=class_weight)
    forest.fit(X, y)
    
    features = {}
    for feature, importance in zip(feature_names, forest.feature_importances_):
        features[feature] = importance #add the name/value pair 

    importances = pd.DataFrame.from_dict(features, orient='index').rename(columns={0: 'Gini-importance'})
    
    importances.sort_values(by='Gini-importance').plot(kind='bar')
    plt.xlabel('Feature')
    plt.ylabel('Gini-Importance')
    plt.title('Feature Importance via RandomForestClassifer')
    plt.show()
    return importances

importances = estimate_feature_importances(X=X, y=y, feature_names=X.columns)

In [ ]:
importances = estimate_feature_importances(X=X, y=y, feature_names=X.columns, class_weight='balanced')

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

def evaluate_pipeline(X, y, param_grid, pipeline, scoring='accuracy', random_state=12):
    """ Conduct a GridSearchCV with supplied pipeline and parameters, use reserved data for output metrics"""
    
    # Reserve a portion of the data, to reduce overfitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_state)
    
    # Initiate GridSearchCV, with 5-fold cross validtation schema, using selected scoring method
    grid = GridSearchCV(pipeline, cv=5, n_jobs=1, param_grid=param_grid, scoring=scoring)
   
    # Fit the Grid to the training data
    grid.fit(X=X_train, y=y_train)
    
    # Inspect the output of the gridsearch
    print("   Scoring Methodology:", scoring)
    print("            Best Score:", grid.best_score_)
    print("        Best Estimator:")
    print(grid.best_estimator_)
    
    # Use the fitted grid to predict on the test data
    y_pred = grid.predict(X_test)
    print("----------------- Test Set Results -----------------")
    print("              Accuracy:", accuracy_score(y_pred=y_pred, y_true=y_test))
    print(" Classification Report:\n", classification_report(y_pred=y_pred, y_true=y_test))
    
    # Return the fitted grid for use, inspection, and storage
    return grid

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('select', SelectKBest()),
    ('approximate', None),
    ('classify', None)
])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier   

knn_parameters = [
    {
        'select__k': [2, 4, 10, 'all'],
        'classify': [KNeighborsClassifier()],
        'classify__weights': ['uniform', 'distance'],
        'classify__algorithm': ['auto'],
        'classify__n_neighbors': [3, 4, 5],
        'classify__n_jobs': [4]
    }
]

grid = evaluate_pipeline(X, y, knn_parameters, pipeline, scoring='precision')

In [ ]:
from sklearn.svm import SVC

svc_parameters = [
    {
        'select__k': [10, 'all'],
        'classify': [SVC()],
        'classify__class_weight': ['balanced'],
        'classify__kernel': ['linear', 'rbf'],
        'classify__gamma': ['auto'],
        'classify__C': [0.1, 1.0]
    }
]

grid = evaluate_pipeline(X, y, svc_parameters, pipeline, scoring='precision')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_parameters = [
    {
        'select__k': [2,4,8,'all'],
        'classify': [RandomForestClassifier()],
        'classify__class_weight': ['balanced', 'balanced_subsample', None],
        'classify__criterion': ['gini', 'entropy'],
        'classify__max_features': ['auto', 'log2'],
        'classify__max_depth': [2, 10, 15, None],
        'classify__n_jobs': [4]
    }
]

grid = evaluate_pipeline(X, y, random_forest_parameters, pipeline, scoring='precision')

In [ ]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

kernel_approximation_parameters = [
    {
        'approximate': [RBFSampler()],
        'approximate__gamma': [0.1, 1, 10],
        'approximate__n_components': [100],
        'classify': [SGDClassifier()],
        'classify__max_iter': [5],
        'classify__class_weight': ['balanced'],
        'classify__n_jobs': [4]
    }
]

grid = evaluate_pipeline(X, y, kernel_approximation_parameters, pipeline, scoring='precision')